# Aprimorando a MLP com Grid Search para escolher o melhor número de camadas intermediárias nas Equações de Mackey-Glass

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 100

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

from tensorflow import keras

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

### 1.3 Bibliotecas dos sistemas caóticos

In [2]:
import sys 
sys.path.insert(0, '../../../../scripts')

import timeseries as times
import mackeyglassequations as mgeq

## 2. Gerando a série temporal

In [3]:
t_inicial = 0
t_final = 5000
tau = 22
n = 10
gamma = 0.1
beta = 0.2
theta = 1

In [4]:
macglass = mgeq.MackeyGlass(tau=tau, gamma=gamma, beta=beta, n=n, theta=theta)

In [5]:
solucoes, instantes_temporais = macglass.calcular(t_inicial = t_inicial, t_final = t_final)

Generating, compiling, and loading C code.
Using default integration parameters.


### Série Temporal de Mackey-Glass

In [7]:
fig, ax = plt.subplots()
ax.set_title('Série temporal de 0 a 600 dias da equação de Mackey-Glass para\n' + r'$\tau =$ ' + str(tau) + r', $\beta =$ ' + str(beta) + r', $\gamma =$ ' + str(gamma) + r', $n =$ ' + str(n) + r' e $\theta =$ ' + str(theta) + ' utilizando $P(0) =$ ' + str(0.1*theta))
ax.plot(instantes_temporais, solucoes, color='Gold')

ax.set_ylabel('$P(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(0,600)
    
ax.grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [10]:
K = 4
L = 3
tam_teste = 0.15

In [11]:
x = np.array(solucoes)
x = np.reshape(x, (1, len(x)))
x = x[0]

In [12]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [13]:
serie_temporal.criar_matrizes()

In [14]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste)

## 3. Definindo o modelo para a MLP

### 3.1 Definindo função para criar a MLP

In [33]:
def criar_modelo(input_size, optimizer='Nadam', batch_normalization='OFF', learning_rate=0.001, activation='selu', init_mode='lecun_normal', n_neurons=30,
                 n_hidden_layers=1, loss="mean_squared_error", name="MLP"):
    
    if (optimizer=='Nadam'):
        model_optimizer = keras.optimizers.Nadam()
    
    model_optimizer.learning_rate.assign(learning_rate)
    
    model = keras.Sequential(name=name)
    model.add(keras.layers.Dense(input_size, input_dim=input_size, name="camada_de_entrada", activation = 'linear'))
    if (batch_normalization == 'ON'):
        model.add(keras.layers.BatchNormalization(name="camada_de_batch_normalization"))
    for i in range(0, n_hidden_layers):
        model.add(keras.layers.Dense(n_neurons, input_dim=input_size, activation=activation, kernel_initializer=init_mode, name="camada_intermediaria_"+str(i+1)))
    model.add(keras.layers.Dense(1, activation='linear', name="camada_de_saida"))
    
    model.compile(
        optimizer = model_optimizer,
        loss = loss)
    
    model.build()
    return model

### 3.2 Definindo parâmetros que não serão definidos pelo *Grid Search*

In [17]:
loss = "mean_squared_error"
optimizer = 'Nadam'
batch_size = 4
batch_normalization = 'OFF'
activation = 'tanh'
init_mode = 'glorot_normal'
n_neurons = 5
learning_rate = 0.001
input_size = K

### 3.3 Executando *Grid Search* para saber o melhor número de camadas intermediárias

In [18]:
model_cv = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=batch_size, batch_normalization=batch_normalization,
                          learning_rate=learning_rate, activation=activation, init_mode=init_mode, n_neurons=n_neurons,
                          loss=loss, optimizer=optimizer, input_size=input_size)

In [19]:
param_grid = {'n_hidden_layers' : (1, 2, 3, 4, 5)}

In [20]:
grid = GridSearchCV(estimator=model_cv, param_grid=param_grid, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [21]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


In [22]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.004630 using {'n_hidden_layers': 4}


In [23]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.006590 (0.003658) with: {'n_hidden_layers': 1}
-0.010946 (0.006145) with: {'n_hidden_layers': 2}
-0.006404 (0.004295) with: {'n_hidden_layers': 3}
-0.004630 (0.003092) with: {'n_hidden_layers': 4}
-0.010009 (0.007319) with: {'n_hidden_layers': 5}


### Treino com o melhor modelo

In [ ]:
n_hidden_layers = 

In [34]:
model = criar_modelo(batch_normalization='OFF', learning_rate=0.001, activation='tanh', init_mode='glorot_normal', n_neurons=5, name="MLP-MackeyGlass")

In [35]:
model.summary()

Model: "MLP-1-camada-intermediaria-MackeyGlass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_de_entrada (Dense)    (None, 4)                 20        
_________________________________________________________________
camada_intermediaria (Dense) (None, 5)                 25        
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 6         
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [36]:
X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.1)

In [37]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

In [39]:
history = model.fit(X_treino, y_treino, epochs=100,
                            callbacks=early_stopping, validation_data=(X_val, y_val),
                            batch_size=batch_size)
treinamento = pd.DataFrame(history.history)

Epoch 1/100
955/955 [==============================] - 2s 2ms/step - loss: 0.0324 - val_loss: 0.0041
Epoch 2/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0033 - val_loss: 0.0017
Epoch 3/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0019 - val_loss: 0.0019
Epoch 4/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0018 - val_loss: 0.0014
Epoch 5/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0018 - val_loss: 0.0014
Epoch 6/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0017 - val_loss: 0.0017
Epoch 7/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0017 - val_loss: 0.0013
Epoch 8/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0017 - val_loss: 0.0013
Epoch 9/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0016 - val_loss: 0.0014
Epoch 10/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0016 - val_lo

## Teste com o melhor modelo

In [40]:
y_pred = model.predict(X_teste)

### Gráfico

In [71]:
fig, ax = plt.subplots(figsize=(6.8,6))

ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_teste, color='Gold', label='Valor real')
ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_pred, color='Silver', label='MLP')

ax.set_title('Comparação da predição da MLP com o valor real da equação de Mackey-Glass\n utilizando a rede neural ótima no conjunto de teste')
ax.set_ylabel('$P(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(5000*(1-tam_teste), 5000)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()
fig.savefig("../../../../images/mlp-aprimorada/performance/mlp-aprimorada-vs-mackeyglass.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Erro Quadrático Médio

In [1]:
from sklearn.metrics import mean_squared_error

In [ ]:
mse = mean_squared_error(y_teste, y_pred)
print("MSE = " + str(mse))

### Salvando o Modelo

In [70]:
model.save("../../../../models/mlp-aprimorada/mlp-aprimorada-mackeyglass.h5", include_optimizer=True)